In [1]:
import numpy as np 
import pandas as pd 

In [2]:
# p = '/kaggle/input/marchprobability/'
# ads = ['Cities', 'Conferences', 'MMasseyOrdinals', 'MNCAATourneySeedRoundSlots', 'MTeamCoaches', 'SampleSubmissionStage1', 'SampleSubmissionStage2', 'SeedBenchmarkStage1']
# mfds = ['ConferenceTourneyGames', 'GameCities', 'NCAATourneyCompactResults', 'NCAATourneyDetailedResults', 'NCAATourneySeeds', 'NCAATourneySlots', 'RegularSeasonCompactResults', 'RegularSeasonDetailedResults', 'Seasons', 'SecondaryTourneyCompactResults', 'SecondaryTourneyTeams', 'TeamConferences', 'TeamSpellings', 'Teams']
# notused = []

In [3]:
import os

# Walk through the directory and filter for paths containing 'marchprobability'
for dirname, _, filenames in os.walk('/kaggle/input'):
    if 'marchprobability' in dirname:  # Only consider directories that contain 'marchprobability'
        for filename in filenames:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marchprobability/Conferences.csv
/kaggle/input/marchprobability/WNCAATourneyDetailedResults.csv
/kaggle/input/marchprobability/WRegularSeasonCompactResults.csv
/kaggle/input/marchprobability/MNCAATourneySeedRoundSlots.csv
/kaggle/input/marchprobability/MRegularSeasonDetailedResults.csv
/kaggle/input/marchprobability/MNCAATourneyCompactResults.csv
/kaggle/input/marchprobability/MGameCities.csv
/kaggle/input/marchprobability/WSecondaryTourneyCompactResults.csv
/kaggle/input/marchprobability/WGameCities.csv
/kaggle/input/marchprobability/MSeasons.csv
/kaggle/input/marchprobability/WNCAATourneySlots.csv
/kaggle/input/marchprobability/MSecondaryTourneyTeams.csv
/kaggle/input/marchprobability/Cities.csv
/kaggle/input/marchprobability/MTeamSpellings.csv
/kaggle/input/marchprobability/MRegularSeasonCompactResults.csv
/kaggle/input/marchprobability/MMasseyOrdinals.csv
/kaggle/input/marchprobability/MSecondaryTourneyCompactResults.csv
/kaggle/input/marchprobability/WTeams.csv
/kagg

1. Two Seperate Pipelines
* Men’s Model Pipeline (D1 Men's teams)
* Women’s Model Pipeline (D1 Women's teams)
* Combine into one prediction file after training and testing.

2. Team-Level Dataset (Both men and women)
Season long
* Offensive/Defensive efficiences
* Average ELO? of conference teams
* Adjusted metrics (KenPom, Massey)
* Strength of schedule
* Returning production (minutes, points)
* Record against Top 25/Quadrant 1 teams
* How close are their wins/losses (point differential)
* Do they play up or down to opponents?

Experience vs Talent
* Returning Production
* Recruiting Class Rankings -> https://n.rivals.com/team_rankings/2024/all-teams/basketball
* Veteran backcourt???
* High assist to turnover ratio (especially for women)

Momentum & last Ten games
* Win streaks
* Recent offensive efficiency
* Peak at the right time?

Style of play
* Tempo differential
* Defense vs 3PT shooting
* Rebounding Margin

Clutch Performance and Close Games
* Record in games decided by 5 points or less
* Freethrow % (Late game?)
* Turnover % in the last 5 minutes

Coaching Impact
* Tournament Experience
* Has the coach won in the tournament before?
* Has the coach played in the tournament before?
* Has the coach outperformed before?
* Use the coaching as a tie breaker

**Matchup Dataset**
For each possible matchup
* Difference features: Team A - Team B
      ELO_diff = TEAMA_ELO - TEAMB_ELO
      Offensive Efficiency Diff
      Deffensive Efficienct Diff
      Seed difference?
* Ratio features (TeamA_Eff / TeamB_Eff)
* Interaction features (tempo)
* Binary flags: same conference, rematch indicators


Overfitting to past seasons - > 	Use cross-validation by season or KFold.

Data gaps for Women's advanced stats - > Focus on core stats (FG%, TO%, rebounding), and use public rankings and seeds to fill in gaps.

Time constraints before submission - > Start simple → Add complexity as time allows.

Probabilities calibration for Brier Score - > Use calibration techniques after training.

**Model Training**
Use historical matchup outcomes (including past tournaments & regular season)
Models:
Start with Logistic Regression (simple, interpretable)
Move to XGBoost / LightGBM for better performance with tabular data
Labels:
Binary win/loss (1 if Team A wins, 0 if Team B wins)
Output:
Probability Team A wins (calibrated)

Probability Calibration
Brier score requires well-calibrated outputs!
Apply:
Platt Scaling (sigmoid/logistic calibration)
Isotonic Regression (non-parametric)
Validate calibration using reliability diagrams.

In [4]:
# Team Performance Metric (Standard)
# Pass the file path when calling the function
file_path = '/kaggle/input/marchprobability/MNCAATourneyDetailedResults.csv'

def team_performance(file_path):

    results_men = pd.read_csv(file_path)
    
    # Calculate Shooting Efficiencies
    results_men['W_Effective_FG%'] = (results_men['WFGM'] + 0.5 * results_men['WFGM3']) / results_men['WFGA']
    results_men['L_Effective_FG%'] = (results_men['LFGM'] + 0.5 * results_men['LFGM3']) / results_men['LFGA']
    
    # Calculate Free Throw Efficiency
    results_men['WFT_efficiency'] = results_men['WFTM'] / results_men['WFTA']
    results_men['LFT_efficiency'] = results_men['LFTM'] / results_men['LFTA']
    
    results_men['WFT_Rate'] = results_men['WFTA'] / results_men['WFGA']
    results_men['LFT_Rate'] = results_men['LFTA'] / results_men['LFGA']
    
    # Rebound Ratios
    results_men['WOffReb%'] = results_men['WOR'] / (results_men['WOR'] + results_men['LDR'])
    results_men['LOffReb%'] = results_men['LOR'] / (results_men['LOR'] + results_men['WDR'])
    
    # Turnover Percentage
    results_men['WTO%'] = (100 * results_men['WTO']) / (results_men['WFGA'] + 0.44 * results_men['WFTA'] + results_men['WTO'])
    results_men['LTO%'] = (100 * results_men['LTO']) / (results_men['LFGA'] + 0.44 * results_men['LFTA'] + results_men['LTO'])
    
    # Number of Possessions for each team
    results_men['WPossessions'] = (results_men['WFGA'] - results_men['WOR']) + results_men['WTO'] + (0.44 * results_men['WFTA'])
    results_men['LPossessions'] = (results_men['LFGA'] - results_men['LOR']) + results_men['LTO'] + (0.44 * results_men['LFTA'])
    
    # Offensive and Defensive Efficiency
    results_men['WOffensiveRating'] = (results_men['WScore'] / results_men['WPossessions']) * 100
    results_men['WDefenseiveRating'] = (results_men['LScore'] /  results_men['LPossessions']) * 100
    
    results_men['LOffensiveRating'] = (results_men['LScore'] / results_men['LPossessions']) * 100
    results_men['LDefenseiveRating'] = (results_men['WScore'] /  results_men['WPossessions']) * 100
                                       
    # Home/Away/Neutral Location Features
    results_men['is_home_game'] = results_men['WLoc'].apply(lambda x: 1 if x == 'home' else 0)
    results_men['is_neutral_game'] = results_men['WLoc'].apply(lambda x: 1 if x == 'neutral' else 0)
    
    # Overtime Flag
    results_men['is_overtime'] = results_men['NumOT'].apply(lambda x: 1 if x > 0 else 0)

    return results_men

In [5]:
# Ensure you call the function and store the updated DataFrame
updated_results = team_performance('/kaggle/input/marchprobability/MNCAATourneyDetailedResults.csv')

# Display the updated results (with new features)
updated_results.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LTO%,WPossessions,LPossessions,WOffensiveRating,WDefenseiveRating,LOffensiveRating,LDefenseiveRating,is_home_game,is_neutral_game,is_overtime
0,2003,134,1421,92,1411,84,N,1,32,69,...,15.683814,78.44,78.64,117.287098,106.815870,106.815870,117.287098,0,0,1
1,2003,136,1112,80,1436,51,N,0,31,66,...,20.218839,77.16,76.08,103.680664,67.034700,67.034700,103.680664,0,0,0
2,2003,136,1113,84,1272,71,N,0,31,59,...,13.297872,67.68,70.24,124.113475,101.082005,101.082005,124.113475,0,0,0
3,2003,136,1141,79,1166,73,N,0,29,53,...,23.734177,71.00,74.48,111.267606,98.012889,98.012889,111.267606,0,0,0
4,2003,136,1143,76,1301,74,N,1,27,64,...,17.766497,69.12,68.80,109.953704,107.558140,107.558140,109.953704,0,0,1


In [6]:
# NBA PIE (Player Impact Estimate) Rating
def calculate_pie(row):
    # PIE Formula
    W_total_stats = (row['WFGM'] + row['WFTA'] + row['WFGM3'] + row['WAst'] + row['WOR'] + row['WDR'] + row['WStl'] + row['WBlk'])
    L_total_stats = (row['LFGM'] + row['LFTA'] + row['LFGM3'] + row['LAst'] + row['LOR'] + row['LDR'] + row['LStl'] + row['LBlk'])

    total_game_stats = W_total_stats + L_total_stats

    if total_game_stats > 0:
        W_pie = W_total_stats / total_game_stats
        L_pie = L_total_stats / total_game_stats
    else:
        W_pie = 0
        L_pie = 0
    
    return pd.Series([W_pie, L_pie])

In [7]:
# Apply the PIE calculation function to each row in updated_results and create new columns
updated_results[['W_PIE', 'L_PIE']] = updated_results.apply(calculate_pie, axis=1)

"""
THE CUMULATIVE PART MAY NEED WORK (assign by Team ID!!!)
"""
# Cumulative PIE for Winning Teams (rolling over season)
updated_results['W_Cumulative_PIE'] = updated_results.groupby('WTeamID')['W_PIE'].cumsum()

# Cumulative PIE for Losing Teams (rolling over season)
updated_results['L_Cumulative_PIE'] = updated_results.groupby('LTeamID')['L_PIE'].cumsum()

# Cumulative PIE for each season's team performance
updated_results['W_Cumulative_PIE_season'] = updated_results.groupby(['Season', 'WTeamID'])['W_PIE'].cumsum()
updated_results['L_Cumulative_PIE_season'] = updated_results.groupby(['Season', 'LTeamID'])['L_PIE'].cumsum()

# Display the updated results with the new features
updated_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDefenseiveRating,is_home_game,is_neutral_game,is_overtime,W_PIE,L_PIE,W_Cumulative_PIE,L_Cumulative_PIE,W_Cumulative_PIE_season,L_Cumulative_PIE_season
0,2003,134,1421,92,1411,84,N,1,32,69,...,117.287098,0,0,1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
1,2003,136,1112,80,1436,51,N,0,31,66,...,103.680664,0,0,0,0.605263,0.394737,0.605263,0.394737,0.605263,0.394737
2,2003,136,1113,84,1272,71,N,0,31,59,...,124.113475,0,0,0,0.525210,0.474790,0.525210,0.474790,0.525210,0.474790
3,2003,136,1141,79,1166,73,N,0,29,53,...,111.267606,0,0,0,0.506494,0.493506,0.506494,0.493506,0.506494,0.493506
4,2003,136,1143,76,1301,74,N,1,27,64,...,109.953704,0,0,1,0.506224,0.493776,0.506224,0.493776,0.506224,0.493776


In [8]:
updated_results.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LDefenseiveRating,is_home_game,is_neutral_game,is_overtime,W_PIE,L_PIE,W_Cumulative_PIE,L_Cumulative_PIE,W_Cumulative_PIE_season,L_Cumulative_PIE_season
0,2003,134,1421,92,1411,84,N,1,32,69,...,117.287098,0,0,1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
1,2003,136,1112,80,1436,51,N,0,31,66,...,103.680664,0,0,0,0.605263,0.394737,0.605263,0.394737,0.605263,0.394737
2,2003,136,1113,84,1272,71,N,0,31,59,...,124.113475,0,0,0,0.525210,0.474790,0.525210,0.474790,0.525210,0.474790
3,2003,136,1141,79,1166,73,N,0,29,53,...,111.267606,0,0,0,0.506494,0.493506,0.506494,0.493506,0.506494,0.493506
4,2003,136,1143,76,1301,74,N,1,27,64,...,109.953704,0,0,1,0.506224,0.493776,0.506224,0.493776,0.506224,0.493776


In [9]:
# Dean Oliver Team Four Factor Rating
def calculateDOFF(row):
    W_DeanOliverFourFactor = (
        0.4 * row['W_Effective_FG%']
        - 0.25 * row['WTO%']
        + 0.2 * row['WOffReb%']
        - 0.15 * row['WFT_Rate']
    )
    L_DeanOliverFourFactor = (
        0.4 * row['L_Effective_FG%']
        - 0.25 * row['LTO%']
        + 0.2 * row['LOffReb%']
        - 0.15 * row['LFT_Rate']
    )
    
    Net_DeanOliverFourFactor = W_DeanOliverFourFactor - L_DeanOliverFourFactor
    
    return pd.Series([W_DeanOliverFourFactor, L_DeanOliverFourFactor, Net_DeanOliverFourFactor])

In [10]:
# Apply your function to the updated DataFrame
updated_results[['W_DeanOliverFourFactor', 'L_DeanOliverFourFactor', 'Net_DeanOliverFourFactor']] = updated_results.apply(calculateDOFF, axis=1)

# Display the updated DataFrame with the new DOFF columns
updated_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,is_overtime,W_PIE,L_PIE,W_Cumulative_PIE,L_Cumulative_PIE,W_Cumulative_PIE_season,L_Cumulative_PIE_season,W_DeanOliverFourFactor,L_DeanOliverFourFactor,Net_DeanOliverFourFactor
0,2003,134,1421,92,1411,84,N,1,32,69,...,1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,-3.017812,-3.709061,0.691249
1,2003,136,1112,80,1436,51,N,0,31,66,...,0,0.605263,0.394737,0.605263,0.394737,0.605263,0.394737,-4.300473,-4.897252,0.596780
2,2003,136,1113,84,1272,71,N,0,31,59,...,0,0.525210,0.474790,0.525210,0.474790,0.525210,0.474790,-2.659422,-3.119796,0.460374
3,2003,136,1141,79,1166,73,N,0,29,53,...,0,0.506494,0.493506,0.506494,0.493506,0.506494,0.493506,-5.249799,-5.690358,0.440559
4,2003,136,1143,76,1301,74,N,1,27,64,...,1,0.506224,0.493776,0.506224,0.493776,0.506224,0.493776,-3.511950,-4.217815,0.705865


In [11]:
# Sully Factor 4
def calculateSFF():
    W_SullyFactorFour = (
        0.5 * row['W_Effective_FG%']
        - 0.3 * row['WTO%']
        + 0.15 * row['WOffReb%']
        - 0.5 * row['WFT_Rate']
    )
    L_SullyFactorFour = (
        0.5 * row['L_Effective_FG%']
        - 0.3 * row['LTO%']
        + 0.15 * row['LOffReb%']
        - 0.5 * row['LFT_Rate']
    )
    
    Net_SullyFactorFour = SullyFactorFour - L_SullyFactorFour
    
    return pd.Series([W_SullyFactorFour, L_SullyFactorFour, Net_SullyFactorFour])

In [12]:
updated_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,is_overtime,W_PIE,L_PIE,W_Cumulative_PIE,L_Cumulative_PIE,W_Cumulative_PIE_season,L_Cumulative_PIE_season,W_DeanOliverFourFactor,L_DeanOliverFourFactor,Net_DeanOliverFourFactor
0,2003,134,1421,92,1411,84,N,1,32,69,...,1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,-3.017812,-3.709061,0.691249
1,2003,136,1112,80,1436,51,N,0,31,66,...,0,0.605263,0.394737,0.605263,0.394737,0.605263,0.394737,-4.300473,-4.897252,0.596780
2,2003,136,1113,84,1272,71,N,0,31,59,...,0,0.525210,0.474790,0.525210,0.474790,0.525210,0.474790,-2.659422,-3.119796,0.460374
3,2003,136,1141,79,1166,73,N,0,29,53,...,0,0.506494,0.493506,0.506494,0.493506,0.506494,0.493506,-5.249799,-5.690358,0.440559
4,2003,136,1143,76,1301,74,N,1,27,64,...,1,0.506224,0.493776,0.506224,0.493776,0.506224,0.493776,-3.511950,-4.217815,0.705865


In [13]:
"""
Because we will not know the exact seeding prior to the the end of 
the competition for the matchups we are going to predict. I will use 
other stats as a proxy for seeding.
"""
# Win-Loss Record
# Point differential 
# Home/Away Record
# Team rankings or Elo ratings
# Strength of schedule (how tough their opponents were during the season)
# Quality wins
# Conference championships
# Performance in key metrics like RPI (Ratings Percentage Index), NET (NCAA Evaluation Tool), and others.

# df['win_percentage'] = df['wins'] / (df['wins'] + df['losses'])
# df['synthetic_seed'] = df['win_percentage'].rank(method='min', ascending=False).astype(int)

'\nBecause we will not know the exact seeding prior to the the end of \nthe competition for the matchups we are going to predict. I will use \nother stats as a proxy for seeding.\n'

In [14]:
# NET (NCAA Evaluation Tool)
    # Winning Percentage
    # Strength of schedule
    # Game Location
    # Scoring Margin
    # Net efficiency metrics

In [15]:
#AP Poll and Coaches Poll
#KenPom
#BPI (Basketball Power Index)

In [16]:
# Team-Level Rankings
    # Strength of schedule
    # Record against Top 25/Quadrant 1 teams
    # Average ELO? of conference teams
    # Adjusted metrics (KenPom, Massey)
    # Returning production (minutes, points)
    # How close are their wins/losses (point differential)
    # Do they play up or down to opponents?

# Focus for right now

    # Seeds
        # seeding = pd.read_csv('')

In [17]:
# Import men's and women's team
# Teamlistmen = pd.read_csv('/kaggle/input/marchprobability/MTeams.csv')
# Teamlistwomen = pd.read_csv('/kaggle/input/marchprobability/WTeams.csv')

# print(Teamlistmen.head())
# print(Teamlistwomen.head())

In [18]:
#Location Data (Feature Engineering)

# men_game_cities = pd.read_csv('/kaggle/input/marchprobability/MGameCities.csv')
# women_game_cities = pd.read_csv('/kaggle/input/marchprobability/WGameCities.csv')
# cities = pd.read_csv('/kaggle/input/marchprobability/Cities.csv')

# print(men_game_cities.head())
# print(women_game_cities.head())
# print(cities.head())

In [19]:
# # Example region assignment (super simple version)
# def assign_region(state):
#     west = ['CA', 'OR', 'WA', 'NV']
#     midwest = ['IL', 'IN', 'OH', 'MI']
#     south = ['TX', 'GA', 'FL']
#     northeast = ['NY', 'PA', 'MA']

#     if state in west:
#         return 'West'
#     elif state in midwest:
#         return 'Midwest'
#     elif state in south:
#         return 'South'
#     elif state in northeast:
#         return 'Northeast'
#     else:
#         return 'Other'

# men_games['Region'] = men_games['State'].apply(assign_region)
# women_games['Region'] = women_games['State'].apply(assign_region)


In [20]:
# # Count how often games are played in each city
# city_freq = men_games['City'].value_counts().to_dict()

# # Add that feature
# men_games['CityFrequency'] = men_games['City'].map(city_freq)
# women_games['CityFrequency'] = women_games['City'].map(city_freq)
